# Caltech Machine Learning Homework # 6

In [2]:
import numpy as np
from sklearn.linear_model import Perceptron
import random
import math
import matplotlib.pyplot as plt
from typing import List
from itertools import product
import scipy.special
from scipy import optimize
import scipy.optimize as spo
from sympy import Symbol, Derivative

def dbg():
    import pdb; pdb.set_trace()

Instructions: https://work.caltech.edu/homework/hw6.pdf

Answers: http://work.caltech.edu/homework/hw6_sol.pdf

## Overfitting and Deterministic Noise

![](imgs/overfitting.png)

**[b]**

## Regularization with Weight Decay

![](imgs/regdecay1.png)

In [3]:
train = np.loadtxt('data/hw5/in.dta.txt')
test = np.loadtxt('data/hw5/out.dta.txt')

X_train = train[:,:-1]
Y_train = train[:,2]
N_train = X_train[:, 0].size

X_test = test[:,:-1]
Y_test = test[:,2]
N_test = X_test[:, 0].size

def theta(X):
    assert(X.shape == (2,))
    x1, x2 = X
    return np.array([1, x1, x2, x1 * x1, x2 * x2, x1 * x2, np.abs(x1-x2), np.abs(x1+x2)])

# Non-linear Transformation
Z_train = np.apply_along_axis(theta, 1, X_train)
Z_test = np.apply_along_axis(theta, 1, X_test)

# Linear Regression
X_dagger = np.dot(np.linalg.inv(np.dot(Z_train.T, Z_train)), Z_train.T)
W = np.dot(X_dagger, Y_train)

# In-sample Error
preds_train = np.sign(np.dot(Z_train, W))
E_IN = sum(preds_train != Y_train) / N_train

# Out-of-sample Error
preds_test = np.sign(np.dot(Z_test, W))
E_OUT = sum(preds_test != Y_test) / N_test

print(f"E_IN is {E_IN}")
print(f"E_OUT is {E_OUT}")


E_IN is 0.02857142857142857
E_OUT is 0.084


**[a]**

![](imgs/regdecay2.png)

In [5]:
k = -3
lambd = 10 ** k

# Linear Regression with Regularization
X_dagger_reg = np.dot(np.linalg.inv(np.dot(Z_train.T, Z_train) + lambd * np.identity(W.size) ), Z_train.T)
W_reg = np.dot(X_dagger_reg, Y_train)

# In-sample Error
preds_train_reg = np.sign(np.dot(Z_train, W_reg))
E_IN_REG = sum(preds_train_reg != Y_train) / N_train

# Out-of-sample Error
preds_test_reg = np.sign(np.dot(Z_test, W_reg))
E_OUT_REG = sum(preds_test_reg != Y_test) / N_test

print(f"E_IN_REG is {E_IN_REG}")
print(f"E_OUT_REG is {E_OUT_REG}")

E_IN_REG is 0.02857142857142857
E_OUT_REG is 0.08


Not much happening here with that small a level of k

**[d]**

![](imgs/regdecay3.png)

In [7]:
k = 3
lambd = 10 ** k

# Linear Regression with Regularization
X_dagger_reg = np.dot(np.linalg.inv(np.dot(Z_train.T, Z_train) + lambd * np.identity(W.size) ), Z_train.T)
W_reg = np.dot(X_dagger_reg, Y_train)

# In-sample Error
preds_train_reg = np.sign(np.dot(Z_train, W_reg))
E_IN_REG = sum(preds_train_reg != Y_train) / N_train

# Out-of-sample Error
preds_test_reg = np.sign(np.dot(Z_test, W_reg))
E_OUT_REG = sum(preds_test_reg != Y_test) / N_test

print(f"E_IN_REG is {E_IN_REG}")
print(f"E_OUT_REG is {E_OUT_REG}")

E_IN_REG is 0.37142857142857144
E_OUT_REG is 0.436


**[e]**

![](imgs/regdecay4.png)

In [10]:
def run_with_k(k):
    lambd = 10 ** k
    
    # Linear Regression with Regularization
    X_dagger_reg = np.dot(np.linalg.inv(np.dot(Z_train.T, Z_train) + lambd * np.identity(W.size) ), Z_train.T)
    W_reg = np.dot(X_dagger_reg, Y_train)

    # In-sample Error
    preds_train_reg = np.sign(np.dot(Z_train, W_reg))
    E_IN = sum(preds_train_reg != Y_train) / N_train

    # Out-of-sample Error
    preds_test_reg = np.sign(np.dot(Z_test, W_reg))
    E_OUT = sum(preds_test_reg != Y_test) / N_test
    
    return E_OUT

print([run_with_k(k)for k in range(-2,3)])

[0.084, 0.056, 0.092, 0.124, 0.228]


Smallest `E_OUT` is $0.056$ at index `1`, so the answer is $k = -1$ **[d]**

![](imgs/regdecay5.png)

We know that $k=3$ is over-regularizing, and $k=-3$ is under-regularizing, so we can limit ourselves to the integer values between $-3$ and $3$:

In [16]:
print([run_with_k(k)for k in range(-3,4)])

[0.08, 0.084, 0.056, 0.092, 0.124, 0.228, 0.436]


So the smallest E_OUT is indeed achieved at $k=-1$, and it's closest to answer **[b]**

![](imgs/regpoly.png)

We have:

$$
H(Q, C, Q_{0}) = {h | h(x) = \textbf w^{T} \textbf z \in H_{Q};w_{q} = C for \  q \geqslant Q_{0}}
$$

**Expanding [a]:**
$$
H(10, 0, 3) = {h | h(x) = \textbf w^{T} \textbf z \in H_{10};w_{q} = 0 for \  q \geqslant 3}
$$

$$
H(10, 0, 4) = {h | h(x) = \textbf w^{T} \textbf z \in H_{10};w_{q} = 0 for \  q \geqslant 4}
$$

This should be essentially a third order polynomial, since all terms $geqslant 4$ will go to zero under the constraint.


Hence [a] is incorrect

**Expanding [b]:**
$$
H(10, 1, 3) = {h | h(x) = \textbf w^{T} \textbf z \in H_{10};w_{q} = 1 for \  q \geqslant 3}
$$

$$
H(10, 1, 4) = {h | h(x) = \textbf w^{T} \textbf z \in H_{10};w_{q} = 1 for \  q \geqslant 4}
$$

There is no effective constraint here at all, so the union of the two should be effectively $\textbf H_{10}$.

Hence [b] is incorrect as well.

**Expanding [c]:**
[c] is the same expansion as [a] above.

So this is the intersection of a third-order with a second-order polynomial, which yields a second-order polynomial

Hence **[c]** looks correct!



---

Let's double-check by looking at [d]:

The two Hypothesis sets here look again effectively unconstrained as in [b], so their intersection should be $\textbf H_{10}$ and not $\textbf H_{1}$

Confirming that **[c]** looks like the correct answer.

## Neural Networks

![](imgs/neuralnets1.png)

From Lecture 10:

![](imgs/neuralnets2.png)

Okay, $d$ seems to be the dimensionality or number of inputs in the layer, so our network should look like this:

![](imgs/neuralnets3.png)

Now, also from Lecture 10:

![](imgs/neuralnets4.png)

![](imgs/neuralnets5.jpg)

So including the non-update to $x_{0}^{l-1}$ we get 44 operations, answer **[d]**

![](imgs/neuralnets5.png)

The minimum should be achieved if we arrange all hidden layers as single neuron layers (with $d=1$).

![](imgs/neuralnets6.jpg)

So I believe it should be **[a]**

![](imgs/neuralnets6.png)

Let's make the first hidden layer as large as possible:

![](imgs/neuralnets7.jpg)

That looks promising, but can we go larger?

![](imgs/neuralnets8.jpg)

The general formula should something like this:
    
$$
maximize ((\sum_{h=1}^H d_{h-1\ n_H}(d_{h\ n_H}-1)) + d_{H\ n_H})
$$

where 

$d_{0\ n_H}=10$ for any $n_H$

$1 \leq H \leq \frac{36}{2}=18 $

$2 \leq n_H \leq 36$ (for each $H$) 

$\sum_{h=1}^H d_{h} = 36$


Let's explore that =)

To find all the possible different architectures, [this](https://en.wikipedia.org/wiki/Composition_(combinatorics)) helped greatly!

So knowing that the term for this is a "composition", I stole the following useful recursive algorithm from [here](https://www.geeksforgeeks.org/print-all-combinations-of-points-that-can-compose-a-given-number/):

In [42]:
architectures = [] # Will hold all possible architectures
N = 36 # Number of neurons to distribute amongst hidden layers
MIN_LAYER_SIZE = 2 # Minimum size of a layer (important for composition calculation)
d_0 = 10 # Dimension of input layer

def computeCompositionsRecursively(n, i, arr = [0] * N):
    if (n==0):
        architectures.append([d_0] + arr.copy()[:i])
    elif(n > 0):
        for k in range(MIN_LAYER_SIZE, N+1):
            arr[i] = k
            computeCompositionsRecursively(n - k, i + 1, arr)
            
computeCompositionsRecursively(N, 0)

print(f"Found {len(architectures)} valid \"architectures\" of {N} neurons arranged in layers of at least 2!")

Found 9227465 valid "architectures" of 36 neurons arranged in layers of at least 2!


We can now find the maximum according to the "general formula" above!

In [56]:
maxWeights = 0
# archs = [[2, 2, 4], [2, 3, 4], [2, 4, 4]]

for a in architectures:
    weights = 0
    
    for i in range(1, len(a)):
        d_hminus1 = a[i-1]
        d_h = a[i]
        weights += d_hminus1 * (d_h - 1)
        
    d_H = a[-1]
    weights += d_H
    
    maxWeights = max(weights, maxWeights)
    
print(f"Maximum weights achievable: {maxWeights}")
    
        
        
        

Maximum weights achievable: 510


So the answer is actually **[e]** =)